### Introduction
This notebook stops the deployment of [llama.cpp](https://github.com/ggerganov/llama.cpp) in SAP AI Core, if the deployment is needed any more. <br/>

### Prerequisites
Before running this notebook, please an deployment of llama.cpp have been created in SAP AI Core through [01-deployment.ipynb](01-deployment.ipynb)<br/><br/>

If the configuration and deployment are created through SAP AI Launchpad, please manually update the configuration_id and deployment_id in [env.json](env.json)
```json
{
    "configuration_id": "<YOUR_CONFIGURATION_ID_OF_LLAMA.CPP_SCENARIO>",
    "deployment_id": "<YOUR_DEPLOYMENT_ID_BASED_ON_CONFIG_ABOVE>"
}
```
 
### The high-level flow of this Continuous Deployment process:
- Connect to SAP AI Core via SDK<br/>
- Stop and Delete the deployment<br/>


#### 1.Initiate an SAP AI Core SDK client
- resource_group loaded from [../config.json](../config.json)
- ai_core_sk(service key) loaded from [../config.json](../config.json)

In [1]:
import json, time
from ai_core_sdk.ai_core_v2_client import AICoreV2Client
from ai_core_sdk.models import Status

In [2]:
# load the configuration from ../config.json 
with open("../config.json") as f:
    config = json.load(f)

resource_group = config.get("resource_group", "default")
print( "resource group: ", resource_group)

resource group:  oss-llm


In [3]:
# Initiate an AI Core SDK client with the information of service key
ai_core_sk = config["ai_core_service_key"]
client = AICoreV2Client(base_url=ai_core_sk.get("serviceurls").get("AI_API_URL")+"/v2",
                        auth_url=ai_core_sk.get("url")+"/oauth/token",
                        client_id=ai_core_sk.get("clientid"),
                        client_secret=ai_core_sk.get("clientsecret"),
                        resource_group=resource_group)

#### 2.Stop and delete the deployment
To stop and delete a deployment in SAP AI Core, it requires the corresponding resource_group and configuration_id
- resource_group loaded from [../config.json](../config.json)
- deploment_id of  loaded from [env.json](env.json)

In [4]:
# resource_group: The target resource group to create the deployment
# configuration_id: The target configuration to create the deployment, which is created in ../00-init-config.ipynb 
with open("./env.json") as f:
    env = json.load(f)

deployment_id = env["deployment_id"]
print("deployment id: ", deployment_id)

deployment id:  d2b3f7ffbde78b38


In [5]:
# Helper function to write back the configuration value back to configuration json file
def update_json_file(file_path, key, value):
    # Load the JSON configuration file
    with open(file_path, 'r') as file:
        config = json.load(file)

    # Update the value
    config[key] = value

    # Write the updated configuration back to the file
    with open(file_path, 'w') as file:
        json.dump(config, file, indent=4)
        print(f"{file_path} updated. {key}: {value}")

Stop and delete the deployment

In [6]:
deployment = client.deployment.get(deployment_id, resource_group=resource_group)

# If the deployment status isn't UNKNOWN, then stop it first
if deployment.status != Status.UNKNOWN:
    response = client.deployment.modify(
        deployment_id, target_status=Status.STOPPED, resource_group=resource_group
    )
    interval_s = 20
    status = None
    while status != Status.STOPPED:
        time.sleep(interval_s)
        deployment = client.deployment.get(deployment_id, resource_group=resource_group)
        status = deployment.status
        print("...... killing deployment ......", flush=True)
        print(f"Deployment status: {deployment.status}")

# Delete the deployment
response = client.deployment.delete(
    deployment_id, resource_group=resource_group
)
print(response)

# Remove deployment_id from env.json
update_json_file("env.json", "deployment_id", "")

...... killing deployment ......
Deployment status: Status.RUNNING
...... killing deployment ......
Deployment status: Status.STOPPED
Id: d2b3f7ffbde78b38, Message: Deletion scheduled
env.json updated. deployment_id: 
